In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 8

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()  
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()  
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda()  
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda()  
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda() 
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda() 
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda() 
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda() 
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda() 
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda() 
    xd2[:, 3] = torch.ones(Nb)
    xe1 = torch.rand(Nb, dimension).cuda()  
    xe1[:, 4] = torch.zeros(Nb)
    xe2 = torch.rand(Nb, dimension).cuda()  
    xe2[:, 4] = torch.ones(Nb)
    xf1 = torch.rand(Nb, dimension).cuda() 
    xf1[:, 5] = torch.zeros(Nb)
    xf2 = torch.rand(Nb, dimension).cuda() 
    xf2[:, 5] = torch.ones(Nb)
    xg1 = torch.rand(Nb, dimension).cuda() 
    xg1[:, 6] = torch.zeros(Nb)
    xg2 = torch.rand(Nb, dimension).cuda() 
    xg2[:, 6] = torch.ones(Nb)
    xh1 = torch.rand(Nb, dimension).cuda() 
    xh1[:, 7] = torch.zeros(Nb)
    xh2 = torch.rand(Nb, dimension).cuda() 
    xh2[:, 7] = torch.ones(Nb)
    
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx4[:, 3] = torch.ones(xb1.size()[0])
    dx5 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx5[:, 4] = torch.ones(xb1.size()[0])
    dx6 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx6[:, 5] = torch.ones(xb1.size()[0])
    dx7 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx7[:, 6] = torch.ones(xb1.size()[0])
    dx8 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx8[:, 7] = torch.ones(xb1.size()[0])
    
    uxa1 = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxa2 = (((model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxb1 = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxb2 = (((model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxc1 = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxc2 = (((model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxd1 = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxd2 = (((model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxe1 = (((model(xe1+step_size*dx5) - model(xe1-step_size*dx5))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxe2 = (((model(xe2+step_size*dx5) - model(xe2-step_size*dx5))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxf1 = (((model(xf1+step_size*dx6) - model(xf1-step_size*dx6))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxf2 = (((model(xf2+step_size*dx6) - model(xf2-step_size*dx6))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxg1 = (((model(xg1+step_size*dx7) - model(xg1-step_size*dx7))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxg2 = (((model(xg2+step_size*dx7) - model(xg2-step_size*dx7))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxh1 = (((model(xh1+step_size*dx8) - model(xh1-step_size*dx8))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxh2 = (((model(xh2+step_size*dx8) - model(xh2-step_size*dx8))/step_size/2 * (-1.0) - 0)**2).cuda() 
    
    part_2 = torch.sum((uxa1 + uxa2 + uxb1 + uxb2 + uxc1 + uxc2 + uxd1 + uxd2 + uxe1 + uxe2 + uxf1 + uxf2 + uxg1 + uxg2 + uxh1 + uxh2)[:,0])/xa1.size()[0]
    lambda1 = 1.0
    return part_1 + lambda1 * part_2 / 16

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 16)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:1" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=8, out_features=16, bias=True)
  (linear1): Linear(in_features=16, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=16, bias=True)
  (linear3): Linear(in_features=16, out_features=16, bias=True)
  (linear4): Linear(in_features=16, out_features=16, bias=True)
  (linear5): Linear(in_features=16, out_features=16, bias=True)
  (linear6): Linear(in_features=16, out_features=16, bias=True)
  (linear_out): Linear(in_features=16, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [16]:
traintime = 50000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_8D_Neumann.npy", error_save)
np.save("DRM_relative_error_8D_Neumann.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(338.9942, device='cuda:1')
current relative error is:  tensor(2.3887, device='cuda:1')
current epoch is:  50
current loss is:  tensor(1.9604, device='cuda:1')
current relative error is:  tensor(0.8800, device='cuda:1')
current epoch is:  100
current loss is:  tensor(-18.0431, device='cuda:1')
current relative error is:  tensor(0.5551, device='cuda:1')
current epoch is:  150
current loss is:  tensor(-24.4883, device='cuda:1')
current relative error is:  tensor(0.4100, device='cuda:1')
current epoch is:  200
current loss is:  tensor(-25.5850, device='cuda:1')
current relative error is:  tensor(0.3658, device='cuda:1')
current epoch is:  250
current loss is:  tensor(-30.0512, device='cuda:1')
current relative error is:  tensor(0.3073, device='cuda:1')
current epoch is:  300
current loss is:  tensor(-29.6821, device='cuda:1')
current relative error is:  tensor(0.2888, device='cuda:1')
current epoch is:  350
current loss is:  tensor(-28.9968, de

current epoch is:  3050
current loss is:  tensor(-39.2852, device='cuda:1')
current relative error is:  tensor(0.1379, device='cuda:1')
current epoch is:  3100
current loss is:  tensor(-36.5256, device='cuda:1')
current relative error is:  tensor(0.0912, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(-42.7277, device='cuda:1')
current relative error is:  tensor(0.0854, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(-36.1616, device='cuda:1')
current relative error is:  tensor(0.1426, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(-42.3980, device='cuda:1')
current relative error is:  tensor(0.0978, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(-34.7818, device='cuda:1')
current relative error is:  tensor(0.0843, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-39.5612, device='cuda:1')
current relative error is:  tensor(0.0767, device='cuda:1')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-39.5129, device='cuda:1')
current relative error is:  tensor(0.0501, device='cuda:1')
current epoch is:  6150
current loss is:  tensor(-34.4649, device='cuda:1')
current relative error is:  tensor(0.0521, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(-37.7325, device='cuda:1')
current relative error is:  tensor(0.0729, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(-34.8701, device='cuda:1')
current relative error is:  tensor(0.0921, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(-39.6152, device='cuda:1')
current relative error is:  tensor(0.0855, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(-37.7762, device='cuda:1')
current relative error is:  tensor(0.0795, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(-40.1074, device='cuda:1')
current relative error is:  tensor(0.0545, device='cuda:1')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-39.2553, device='cuda:1')
current relative error is:  tensor(0.0397, device='cuda:1')
current epoch is:  9200
current loss is:  tensor(-39.5718, device='cuda:1')
current relative error is:  tensor(0.0694, device='cuda:1')
current epoch is:  9250
current loss is:  tensor(-37.3020, device='cuda:1')
current relative error is:  tensor(0.0398, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(-36.3017, device='cuda:1')
current relative error is:  tensor(0.0813, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(-38.6322, device='cuda:1')
current relative error is:  tensor(0.0439, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(-38.6693, device='cuda:1')
current relative error is:  tensor(0.0670, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(-37.3757, device='cuda:1')
current relative error is:  tensor(0.0984, device='cuda:1')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-40.2380, device='cuda:1')
current relative error is:  tensor(0.0557, device='cuda:1')
current epoch is:  12200
current loss is:  tensor(-40.7921, device='cuda:1')
current relative error is:  tensor(0.0365, device='cuda:1')
current epoch is:  12250
current loss is:  tensor(-40.6908, device='cuda:1')
current relative error is:  tensor(0.0290, device='cuda:1')
current epoch is:  12300
current loss is:  tensor(-40.1903, device='cuda:1')
current relative error is:  tensor(0.0309, device='cuda:1')
current epoch is:  12350
current loss is:  tensor(-40.5513, device='cuda:1')
current relative error is:  tensor(0.0396, device='cuda:1')
current epoch is:  12400
current loss is:  tensor(-37.6932, device='cuda:1')
current relative error is:  tensor(0.0267, device='cuda:1')
current epoch is:  12450
current loss is:  tensor(-39.0985, device='cuda:1')
current relative error is:  tensor(0.0451, device='cuda:1')
current epoch is:  12500
current loss is:

...

current epoch is:  36150
current loss is:  tensor(-42.2071, device='cuda:1')
current relative error is:  tensor(0.0946, device='cuda:1')
current epoch is:  36200
current loss is:  tensor(-35.1878, device='cuda:1')
current relative error is:  tensor(0.0627, device='cuda:1')
current epoch is:  36250
current loss is:  tensor(-39.3231, device='cuda:1')
current relative error is:  tensor(0.0224, device='cuda:1')
current epoch is:  36300
current loss is:  tensor(-38.1889, device='cuda:1')
current relative error is:  tensor(0.0664, device='cuda:1')
current epoch is:  36350
current loss is:  tensor(-38.6810, device='cuda:1')
current relative error is:  tensor(0.0319, device='cuda:1')
current epoch is:  36400
current loss is:  tensor(-37.8025, device='cuda:1')
current relative error is:  tensor(0.0378, device='cuda:1')
current epoch is:  36450
current loss is:  tensor(-39.8775, device='cuda:1')
current relative error is:  tensor(0.0376, device='cuda:1')
current epoch is:  36500
current loss is:

current epoch is:  39150
current loss is:  tensor(-37.6388, device='cuda:1')
current relative error is:  tensor(0.0372, device='cuda:1')
current epoch is:  39200
current loss is:  tensor(-39.1110, device='cuda:1')
current relative error is:  tensor(0.0214, device='cuda:1')
current epoch is:  39250
current loss is:  tensor(-40.4649, device='cuda:1')
current relative error is:  tensor(0.0566, device='cuda:1')
current epoch is:  39300
current loss is:  tensor(-38.2319, device='cuda:1')
current relative error is:  tensor(0.1153, device='cuda:1')
current epoch is:  39350
current loss is:  tensor(-40.3648, device='cuda:1')
current relative error is:  tensor(0.0402, device='cuda:1')
current epoch is:  39400
current loss is:  tensor(-40.0681, device='cuda:1')
current relative error is:  tensor(0.0344, device='cuda:1')
current epoch is:  39450
current loss is:  tensor(-39.1149, device='cuda:1')
current relative error is:  tensor(0.0429, device='cuda:1')
current epoch is:  39500
current loss is:

current epoch is:  42150
current loss is:  tensor(-43.6448, device='cuda:1')
current relative error is:  tensor(0.0349, device='cuda:1')
current epoch is:  42200
current loss is:  tensor(-38.6189, device='cuda:1')
current relative error is:  tensor(0.0839, device='cuda:1')
current epoch is:  42250
current loss is:  tensor(-36.5608, device='cuda:1')
current relative error is:  tensor(0.0210, device='cuda:1')
current epoch is:  42300
current loss is:  tensor(-42.3279, device='cuda:1')
current relative error is:  tensor(0.0224, device='cuda:1')
current epoch is:  42350
current loss is:  tensor(-44.0670, device='cuda:1')
current relative error is:  tensor(0.0238, device='cuda:1')
current epoch is:  42400
current loss is:  tensor(-37.5970, device='cuda:1')
current relative error is:  tensor(0.0250, device='cuda:1')
current epoch is:  42450
current loss is:  tensor(-39.0710, device='cuda:1')
current relative error is:  tensor(0.0316, device='cuda:1')
current epoch is:  42500
current loss is:

current epoch is:  45150
current loss is:  tensor(-40.9698, device='cuda:1')
current relative error is:  tensor(0.0578, device='cuda:1')
current epoch is:  45200
current loss is:  tensor(-41.2057, device='cuda:1')
current relative error is:  tensor(0.0263, device='cuda:1')
current epoch is:  45250
current loss is:  tensor(-43.3866, device='cuda:1')
current relative error is:  tensor(0.0326, device='cuda:1')
current epoch is:  45300
current loss is:  tensor(-39.4039, device='cuda:1')
current relative error is:  tensor(0.0235, device='cuda:1')
current epoch is:  45350
current loss is:  tensor(-37.0193, device='cuda:1')
current relative error is:  tensor(0.0306, device='cuda:1')
current epoch is:  45400
current loss is:  tensor(-39.9850, device='cuda:1')
current relative error is:  tensor(0.0280, device='cuda:1')
current epoch is:  45450
current loss is:  tensor(-41.4772, device='cuda:1')
current relative error is:  tensor(0.0177, device='cuda:1')
current epoch is:  45500
current loss is:

current epoch is:  48150
current loss is:  tensor(-40.9471, device='cuda:1')
current relative error is:  tensor(0.0252, device='cuda:1')
current epoch is:  48200
current loss is:  tensor(-39.1801, device='cuda:1')
current relative error is:  tensor(0.0236, device='cuda:1')
current epoch is:  48250
current loss is:  tensor(-39.5613, device='cuda:1')
current relative error is:  tensor(0.0530, device='cuda:1')
current epoch is:  48300
current loss is:  tensor(-36.8744, device='cuda:1')
current relative error is:  tensor(0.0185, device='cuda:1')
current epoch is:  48350
current loss is:  tensor(-43.2300, device='cuda:1')
current relative error is:  tensor(0.0432, device='cuda:1')
current epoch is:  48400
current loss is:  tensor(-37.4854, device='cuda:1')
current relative error is:  tensor(0.0523, device='cuda:1')
current epoch is:  48450
current loss is:  tensor(-38.8186, device='cuda:1')
current relative error is:  tensor(0.0231, device='cuda:1')
current epoch is:  48500
current loss is:

In [18]:
x_test = torch.rand(10, 8)
torch.abs(model(x) - u_ex(x))

tensor([[0.0461],
        [0.0138],
        [0.0252],
        ...,
        [0.0635],
        [0.0657],
        [0.0024]], device='cuda:1', grad_fn=<AbsBackward>)